<a href="https://colab.research.google.com/github/HazSyl1/LORA/blob/main/LLM_CHATBOT_MISTRAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-lcx6wros
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-lcx6wros
  Resolved https://github.com/huggingface/transformers to commit 2cc8cf6ce7ae0416561acbb639df4bbc5f409b6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset , Dataset
from peft import LoraConfig,AutoPeftModelForCausalLM ,prepare_model_for_kbit_training , get_peft_model
from transformers import AutoModelForCausalLM ,AutoTokenizer , GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os


In [ ]:
data=load_dataset("tatsu-lab/alpaca",split="train")
data_df=data.to_pandas()
data_df=data_df[:5000]
data_df["text"]=data_df[["input","instruction","output"]].apply(lambda x: "###Human: "+x["instruction"]+ " "+x["input"]+"###Assistant: " +x["output"] ,axis=1)
data=Dataset.from_pandas(data_df)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data['text'][0]

'###Human: Give three tips for staying healthy. ###Assistant: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token=tokenizer.eos_token

quantization_config_loading=GPTQConfig(bits=4,disable_exllama=True,tokenizer=tokenizer)
model=AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
    quantization_config=quantization_config_loading,
    device_map="auto"
)

model.config.use_cache=False
model.config.pretrained_tp=1
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model) #becomes prepared for 4 bit training



tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
peft_config=LoraConfig(
    r=16,lora_alpha=16,lora_dropout=0.05,bias="none",task_type="CAUSAL_LM",target_modules=["q_proj","v_proj"]
)
model=get_peft_model(model,peft_config)

training_arguments=TrainingArguments(
    output_dir="mistral-finetuned-alpaca",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100, #at 100 step it'll give the log of evaluation
    num_train_epochs=1,
    max_steps=100, #here we've 8*1*250= 2000 steps.Meaning it'll work for 2000 steps onlt
    fp16=True,
    push_to_hub=True
)

In [ ]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False, #packing the tokens that dont fit in the max_token_length , in the second batch/iter
    max_seq_length=512
)
trainer.train()

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.454800
200,1.271600


TrainOutput(global_step=250, training_loss=1.336803451538086, metrics={'train_runtime': 1452.0918, 'train_samples_per_second': 1.377, 'train_steps_per_second': 0.172, 'total_flos': 310614365306880.0, 'train_loss': 1.336803451538086, 'epoch': 0.4})

In [ ]:
! cp -r /content/mistral-finetuned-alpaca /content/drive/MyDrive/

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
model.save_pretrained("HazSylvia/Fine-Tuned-Mistral-x")

tokenizer.save_pretrained("HazSylvia/Fine-Tuned-Mistral-x")

('HazSylvia/Fine-Tuned-Mistral-x/tokenizer_config.json',
 'HazSylvia/Fine-Tuned-Mistral-x/special_tokens_map.json',
 'HazSylvia/Fine-Tuned-Mistral-x/tokenizer.model',
 'HazSylvia/Fine-Tuned-Mistral-x/added_tokens.json',
 'HazSylvia/Fine-Tuned-Mistral-x/tokenizer.json')

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your con

In [ ]:
%cd /content/mistral-finetuned-alpaca


/content/mistral-finetuned-alpaca


In [ ]:
!huggingface-cli repo create MISTRAL-FINETUNED-ALPACA-EXP


git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create HazSylvia/MISTRAL-FINETUNED-ALPACA-EXP
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/HazSylvia/MISTRAL-FINETUNED-ALPACA-EXP

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/HazSylvia/MISTRAL-FINETUNED-ALPACA-EXP



In [ ]:
model.push_to_hub("MISTRAL-FINETUNED-ALPACA-xp",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=False)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HazSylvia/MISTRAL-FINETUNED-ALPACA-xp/commit/5ab2769cc08cfe4e605c648b32a5febc2bb68d23', commit_message='basic training', commit_description='', oid='5ab2769cc08cfe4e605c648b32a5febc2bb68d23', pr_url=None, pr_revision=None, pr_num=None)

## INFERENCE

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer=AutoTokenizer.from_pretrained("/content/drive/MyDrive/mistral-finetuned-alpaca")

model=AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/mistral-finetuned-alpaca",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda"
    )
generation_config=GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.2,
    max_new_tokens=200,
    pad_token_id=tokenizer.eos_token_id
)

def chatbot(message):
  input_str="###Human: "+message+" ###Assistant: "
  inputs=tokenizer(input_str,return_tensors="pt").to("cuda")
  outputs=model.generate(**inputs ,generation_config=generation_config)
  return tokenizer.decode(outputs[0],skip_special_tokens=True).replace(input_str,"")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [ ]:
chatbot("What should i eat??")

'\n- Eat a balanced diet with plenty of fruits, vegetables, whole grains, lean protein, and healthy fats.\n- Drink plenty of water and limit your intake of sugary drinks.\n- Avoid processed foods and foods high in saturated fats.\n- Get enough sleep and exercise regularly.\n- Consider taking a multivitamin or supplement if you are not getting enough nutrients from your diet.\n- Consider consulting a nutritionist or dietitian for personalized advice.\n- Consider taking a probiotic to improve gut health.\n- Consider taking a fish oil supplement to improve brain function.\n- Consider taking a vitamin D supplement to improve bone health.\n- Consider taking a vitamin C supplement to improve immune function.\n- Consider taking a vitamin E supplement to improve heart health.\n- Consider taking a vitamin K supplement to improve blood clotting.\n- Consider taking a vitamin B1'